In [120]:
import pandas as pd
import numpy as np

In [141]:
df  = pd.read_pickle("data/data.pkl")
df

,Commune,Votation,Electeurs inscrits,Bulletins rentrés,Participation en %,Bulletins valables,Oui,Non,Oui en %,District,Canton,Pays
0,Aeugst am Albis,29.11.1998 Initiative Droleg,1070.0,487.0,45.5,478.0,167.0,311.0,34.9,Affoltern,Zürich,Suisse
1,Aeugst am Albis,14.06.2015 Initiative sur les bourses d'études,1380.0,706.0,51.2,695.0,186.0,509.0,26.8,Affoltern,Zürich,Suisse
2,Aeugst am Albis,25.09.2016 Loi fédérale sur le renseignement,1400.0,670.0,47.9,659.0,417.0,242.0,63.3,Affoltern,Zürich,Suisse
3,Aeugst am Albis,03.03.1991 Encouragement des transports publics,835.0,321.0,38.4,312.0,128.0,184.0,41.0,Affoltern,Zürich,Suisse
4,Aeugst am Albis,12.02.2017 Réforme de l'imposition des entrepr...,1395.0,759.0,54.4,750.0,318.0,432.0,42.4,Affoltern,Zürich,Suisse
5,Aeugst am Albis,07.03.2010 Recherche sur l'être humain,1262.0,588.0,46.6,579.0,476.0,103.0,82.2,Affoltern,Zürich,Suisse
6,Aeugst am Albis,08.02.2004 Initiative pour l'internement des d...,1135.0,590.0,52.0,578.0,266.0,312.0,46.0,Affoltern,Zürich,Suisse
7,Aeugst am Albis,"22.09.2002 L'or à l'AVS, aux cantons et à la F...",1113.0,610.0,54.8,604.0,325.0,267.0,54.9,Affoltern,Zürich,Suisse
8,Aeugst am Albis,24.11.2002 Contre les abus dans le droit d'asile,1117.0,662.0,59.3,648.0,282.0,366.0,43.5,Affoltern,Zürich,Suisse
9,Aeugst am Albis,01.12.1985 Suppression de la vivisection,683.0,363.0,53.1,358.0,135.0,223.0,37.7,Affoltern,Zürich,Suisse


In [156]:
x = xls.parse(i)
    
x.columns = ['date'] + list(x.columns[1:])
    
base_nb = x[x['date'] == 'Parti 1)'].index[0]



recommand_inter = pd.concat([x.iloc[base_nb:base_nb+1],x.iloc[base_nb+2:base_nb+3],x.dropna()]).transpose()
recommand_inter.columns = range(len(recommand_inter.columns))
recommand_inter[0] = recommand_inter[0]+str(2017-i)
recommand = pd.concat([recommand,recommand_inter])
VVprint(x['date'][:3] == 'No ')
propositions = x[x['date'][:3] == 'No ']
propositions

0    False
1    False
2    False
Name: date, dtype: bool


/anaconda/lib/python3.6/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: '>' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match

In [151]:
xls = pd.ExcelFile('data/Recommandations des partis.xls')
nb_sheets = len(xls.sheet_names)
recommand = pd.DataFrame()

for i in range ((2017-1981 + 1)) :
    
    x = xls.parse(i)
    
    x.columns = ['date'] + list(x.columns[1:])
    
    base_nb = x[x['date'] == 'Parti 1)'].index[0]

    recommand_inter = pd.concat([x.iloc[base_nb:base_nb+1],x.iloc[base_nb+2:base_nb+3],x.dropna()]).transpose()
    recommand_inter.columns = range(len(recommand_inter.columns))
    recommand_inter[0] = recommand_inter[0]+str(2017-i)
    recommand = pd.concat([recommand,recommand_inter])
    
    

month_to_int = {'janvier':'01','février' : '02','févirer' : '02' , 'mars' : '03' , 'avril' : '04' , 'mai' : '05' ,\
               'juin' : '06' , 'juillet':'07' , 'aout' : '08' , 'septembre' : '09' , 'octobre':'10',\
               'novembre':'11' , 'décembre' : '12'}
print('done')



def create_date  (x) :
    split_x = str(x).split(' ')
    print(x,len(str(x).split('.')))
    if (len(str(x).split('.')) > 2 ) :
        print(str(x)[:12])
        return str(x)[:12]
    elif('Parti' not in str(x) and 'nan' not in str(x)) :
        return split_x[0] + '.'+month_to_int.get(split_x[1]) + '.'

    else :
        return 'nan'
    

recommand['date'] = recommand[0].map(lambda x : create_date(x))
                                    
recommand

done
Parti 1)2017 1
12 février 2)2017 1
nan 1
nan 1
21 mai 2)2017 1
24 septembre 2)2017 1
nan 1
nan 1
Parti 1)2016 1
28 février 2)2016 1
nan 1
nan 1
nan 1
5 juin 2)2016 1
nan 1
nan 1
nan 1
nan 1
25 septembre 2)2016 1
nan 1
nan 1
27 novembre 2)2016 1
Parti 1)2015 1
8 mars 2)2015 1
nan 1
14 juin 2)2015 1
nan 1
nan 1
nan 1
Parti 1)2014 1
9 février 2)2014 1
nan 1
nan 1
18 mai 2)2014 1
nan 1
nan 1
nan 1
28 septembre 2)2014 1
nan 1
30 novembre 2)2014 1
nan 1
nan 1
Parti 1)2013 1
3 mars 2)2013 1
nan 1
nan 1
9 juin 2)2013 1
nan 1
22 septembre 2)2013 1
nan 1
nan 1
24 novembre 2)2013 1
nan 1
nan 1
Parti 1)2012 1
11 mars 2)2012 1
nan 1
nan 1
nan 1
nan 1
17 juin 2)2012 1
nan 1
nan 1
23 septembre 2)2012 1
nan 1
nan 1
25 novembre 2)2012 1
Parti 1)2011 1
13 février 2011 2)2011 1
nan 1
nan 1
Parti 1)2010 1
7 mars 2010 2)2010 1
nan 1
nan 1
26.9.2010 2)2010 3
26.9.2010 2)
28.11.2010 2)2010 3
28.11.2010 2
nan 1
nan 1
nan 1
Parti 1)2009 1
8 février 2009 2)2009 1
17 mai 2009 2)2009 1
nan 1
27 septembre 200

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,date
date,Parti 1)2017,NaN,PLR,PDC,PS,UDC,PEV,PVL,PBD,PES,Lega,MCR,NaN,NaN,NaN,NaN,NaN,NaN,nan
Unnamed: 1,12 février 2)2017,no 609,oui,oui,oui,non,oui,oui,oui,oui,-,non,NaN,NaN,NaN,NaN,NaN,NaN,12.02.
Unnamed: 2,NaN,no 610,oui,oui,non,oui,oui,oui,oui,non,-,oui,NaN,NaN,NaN,NaN,NaN,NaN,nan
Unnamed: 3,NaN,no 611,oui,oui,non,oui,non,oui,oui,non,-,oui,NaN,NaN,NaN,NaN,NaN,NaN,nan
Unnamed: 4,21 mai 2)2017,no 612,oui,oui,oui,non,oui,oui,oui,oui,-,non,NaN,NaN,NaN,NaN,NaN,NaN,21.05.
Unnamed: 5,24 septembre 2)2017,no 613,oui,oui,oui,oui,oui,oui,oui,oui,-,oui,NaN,NaN,NaN,NaN,NaN,NaN,24.09.
Unnamed: 6,NaN,no 614,non,oui,oui,non,oui,oui,oui,oui,-,non,NaN,NaN,NaN,NaN,NaN,NaN,nan
T 17.03.01.04,NaN,no 615,non,oui,oui,non,oui,oui,oui,oui,-,non,NaN,NaN,NaN,NaN,NaN,NaN,nan
date,Parti 1)2016,NaN,PLR,PDC,PS,UDC,PEV,PVL,PBD,PES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
Unnamed: 1,28 février 2)2016,no 596,non,oui,non,oui,oui,non,non,non,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.02.


In [115]:
recommand

,0,1,2,3,4,5,6,7,8,9,10,11,year,12,13,14,15,16,17
date,Parti 1),NaN,PLR,PDC,PS,UDC,PEV,PVL,PBD,PES,Lega,MCR,2017,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 1,12 février 2),no 609,oui,oui,oui,non,oui,oui,oui,oui,-,non,2017,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 2,NaN,no 610,oui,oui,non,oui,oui,oui,oui,non,-,oui,2017,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 3,NaN,no 611,oui,oui,non,oui,non,oui,oui,non,-,oui,2017,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 4,21 mai 2),no 612,oui,oui,oui,non,oui,oui,oui,oui,-,non,2017,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 5,24 septembre 2),no 613,oui,oui,oui,oui,oui,oui,oui,oui,-,oui,2017,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 6,NaN,no 614,non,oui,oui,non,oui,oui,oui,oui,-,non,2017,NaN,NaN,NaN,NaN,NaN,NaN
T 17.03.01.04,NaN,no 615,non,oui,oui,non,oui,oui,oui,oui,-,non,2017,NaN,NaN,NaN,NaN,NaN,NaN
date,Parti 1),NaN,PLR,PDC,PS,UDC,PEV,PVL,PBD,PES,NaN,NaN,2016,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 1,28 février 2),no 596,non,oui,non,oui,oui,non,non,non,NaN,NaN,2016,NaN,NaN,NaN,NaN,NaN,NaN
